In [1]:
import argparse, os, random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler


from tensorboardX import SummaryWriter
from sklearn import metrics
import numpy as np
import prismnet.model as arch
from prismnet import train, validate, inference, log_print, compute_saliency, compute_saliency_img, compute_high_attention_region
#compute_high_attention_region

# from prismnet.engine.train_loop import 
from prismnet.model.utils import GradualWarmupScheduler
from prismnet.loader import SeqicSHAPE
from prismnet.utils import datautils

In [2]:
use_cuda = torch.cuda.is_available()

In [3]:
device = 'cuda'

In [4]:
# out dir
data_dir = '/home/hsher/bin/PrismNet/data/rsr_data'
p_name = '441_PUM2_K562'
arch_='PrismNet'
mode = 'pu'
out_dir = '/home/hsher/bin/PrismNet/results/pos_weight/10'
data_path  = os.path.join(data_dir, p_name + ".h5")
identity   = p_name+'_'+arch_+"_"+mode # 441_PUM2_K562_sub4509_PrismNet_pu.metrics
datautils.make_directory(out_dir,"out/")
model_dir  = datautils.make_directory(out_dir,"out/models")
model_path = os.path.join(model_dir, identity+"_{}.pth")

In [5]:
model_path

'/home/hsher/bin/PrismNet/results/pos_weight/10/out/models/441_PUM2_K562_PrismNet_pu_{}.pth'

In [6]:
model = getattr(arch, arch_)(mode=mode)
filename = model_path.format("best")
print("Loading model: {}".format(filename))
#model.load_state_dict(torch.load(filename,map_location='cpu'))
model.load_state_dict(torch.load('../results/pos_weight/10/out/models/441_PUM2_K562_PrismNet_pu_best.pth', map_location = 'cpu'))

Loading model: /home/hsher/bin/PrismNet/results/pos_weight/10/out/models/441_PUM2_K562_PrismNet_pu_best.pth


<All keys matched successfully>

In [7]:
model_path

'/home/hsher/bin/PrismNet/results/pos_weight/10/out/models/441_PUM2_K562_PrismNet_pu_{}.pth'

In [8]:
kwarg = {'num_workers': 3, 'pin_memory': True}

In [9]:
infer_file = '/home/hsher/bin/PrismNet/data/rsr_data/high_prob/441_PUM2_K562.tsv'
test_loader  = torch.utils.data.DataLoader(SeqicSHAPE(infer_file, is_infer=True), \
            batch_size=64, shuffle=False, **kwarg)



Reading inference file from CSV: /home/hsher/bin/PrismNet/data/rsr_data/high_prob/441_PUM2_K562.tsv
Saving into npz.
Saved.
infer data:  64  use_structure:  True


In [28]:
test_loader.dataset

In [24]:
n_pos = 0
n_neg = 0
for x, y in test_loader:
    n_pos += y.sum().detach().item()

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


In [29]:
len(test_loader.dataset)

64

In [30]:
n_pos

63.0

In [10]:
model.to(device)
p_all = inference(model_path, model, device, test_loader)
        

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.hist(p_all)

In [ ]:

compute_high_attention_region(args, model, device, test_loader, identity)

In [13]:
x0[0]

NameError: name 'x0' is not defined

In [14]:
from prismnet.model.smoothgrad import GuidedBackpropSmoothGrad
L = 20
har = ""
# sgrad = SmoothGrad(model, device=device)
sgrad = GuidedBackpropSmoothGrad(model, device=device)
for batch_idx, (x0, y0) in enumerate(test_loader):
    if batch_idx % 50 == 0:
        print(f'HAR at batch {batch_idx}')
    X, Y = x0.float().to(device), y0.to(device).float()
    
    
    output = model(X) # prediction
    prob = torch.sigmoid(output) # probability
    print(prob)
    p_np = prob.to(device='cpu').detach().numpy().squeeze()
    guided_saliency  = sgrad.get_batch_gradients(X, Y)

    attention_region = guided_saliency.sum(dim=3)[:,0,:].to(device='cpu').numpy() # (N, 101, 1)
    N,NS = attention_region.shape # (N, 101)
    for i in range(N):
        
        inr = batch_idx*args.batch_size + i # dataset index
        iar = attention_region[i] # attention score
        ar_score = np.array([ iar[j:j+L].sum() for j in range(NS-L+1)]) # windowing of L
        # import pdb; pdb.set_trace()
        highest_ind = np.argmax(iar) # max window
        har += "{}\t{:.6f}\t{}\t{}\t{}\n".format(inr, p_np[i], highest_ind, highest_ind+L, ','.join(np.char.mod('%.1f', iar)))

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


HAR at batch 0
tensor([[0.9984],
        [0.9966],
        [0.9940],
        [1.0000],
        [0.9995],
        [1.0000],
        [0.9924],
        [0.9970],
        [0.9993],
        [0.9931],
        [0.9999],
        [0.9947],
        [0.9999],
        [0.9990],
        [0.9982],
        [0.9996],
        [0.9999],
        [1.0000],
        [0.9995],
        [1.0000],
        [0.9997],
        [0.9957],
        [0.9986],
        [1.0000],
        [0.9932],
        [1.0000],
        [0.9978],
        [0.9980],
        [1.0000],
        [1.0000],
        [0.9948],
        [0.9993],
        [0.9998],
        [0.9997],
        [0.9998],
        [0.9993],
        [0.9924],
        [0.9999],
        [0.9969],
        [0.9998],
        [0.9978],
        [1.0000],
        [0.9985],
        [0.9998],
        [0.9926],
        [0.9934],
        [0.9929],
        [0.9998],
        [1.0000],
        [1.0000],
        [0.9995],
        [0.9993],
        [0.9990],
        [1.0000],
        [0.99

NameError: name 'args' is not defined

In [ ]:
p_all

In [ ]:
p_np

In [ ]:
import pandas as pd
pd.read_csv('/home/hsher/bin/PrismNet/data/rsr_data/high_prob/441_PUM2_K562.tsv', sep = '\t', header = None)